In [ ]:
import xarray as xr
from xarray import DataArray
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.img_tiles import OSM
from matplotlib.animation import FuncAnimation
import numpy as np
import pandas as pd

# Temperature index models

## Acknowledgment

by Thomas Gölles 2024

## Naive first approach

In [ ]:
degree_day_factor = 8.0  # mm/day/degree celsius
temperature_plus = 3.0  # degree celsius
delta_time = 4  # day

melt = degree_day_factor * temperature_plus * delta_time
print(f"melt of snow for {delta_time} is {melt} mm")

Now lets look at the cumulative melt over 10 days

In [ ]:
n = 11
days = range(1, n)
melt = 0

for day in days:
    melt = melt + degree_day_factor * temperature_plus * delta_time
    print(f"cummulative melt of snow for {day} is {melt:.2f} mm")

ok a bit nicer, but the Temperature is still constant. Now use a random tempearature list form -5 to +10

In [ ]:
n = 11
days = range(1, n)

np.random.seed(42)
temperatures = np.random.uniform(-5, 10, len(days))
melt = 0

for day in days:
    temperature_plus = temperatures[day - 1]
    melt = melt + degree_day_factor * temperature_plus * delta_time
    print(
        f"cummulative melt of snow for {day} with {temperature_plus:.2f}°C is {melt:.2f} mm"
    )

In [ ]:
n = 11
days = range(1, n)

np.random.seed(42)
temperatures = np.random.uniform(-5, 10, len(days))
melt = 0

for day in days:
    current_temperature = temperatures[day - 1]
    temperature_plus = np.maximum(current_temperature, 0)
    melt = melt + degree_day_factor * temperature_plus * delta_time
    print(
        f"cummulative melt of snow for {day} with {current_temperature:.2f}°C is {melt:.2f} mm"
    )

## Better use of Python

If you often reuse a culation but it in a function.
A function starts with "def" then the function name you want to have followed by the input argumetnts in ().

Here I also used so called typehints like degree_day_factor: float. Which means that we expect a floating point number as the degree day factor. In Python this optional but helps a lot in understanding what is expected.

Here I also give a default value of the threshold_temp. This means that if no third argument is given the standard value of 0.0 is used.

In [ ]:
def degree_day_model(
    temperatures: np.ndarray | DataArray,
    degree_day_factor: float,
    threshold_temp: float = 0.0,
) -> np.ndarray | DataArray:
    """
    Estimate daily melt amount using the Degree-Day Model.

    Parameters:
    temperatures (numpy.array): Array of daily mean air temperatures (°C).
    degree_day_factor (float): Degree-day factor (mm °C^-1 day^-1).
    threshold_temp (float): Threshold temperature above which melting occurs (°C). Default is 0.0°C.

    Returns:
    numpy.array: Array of daily melt amounts (mm).
    """
    # Calculate positive temperature differences (temperatures above the threshold)
    positive_temperatures = np.maximum(temperatures - threshold_temp, 0)

    # Calculate daily melt amounts
    daily_melt = positive_temperatures * degree_day_factor

    return daily_melt

In [ ]:
temperatures = np.array([-3.2, -1.1, 0.0, 2.2, 5.1, 3.0, 0.0, -2.0])

In [ ]:
ddf = 8.0
melt_amounts = degree_day_model(temperatures, degree_day_factor=ddf)
melt_amounts

note here we used the default value for threshold_temp.
So the result is an array of values correspondig to daily temperatures.
To make it nicer to work with lets make a pandas dataframe together with the temperatures

In [ ]:
from operator import index


melt = pd.DataFrame(
    melt_amounts,
    index=temperatures,
    columns=[f"melt (mm)"],
)
melt.index.name = "Temperature (°C)"
melt

In [ ]:
melt.plot()

### Exersize

* plot the curve for different degree day factors.
* compare snow and ice: 2.5 to 11.6 mm/(day K)  for snow 6.6 to 20.0 mm/(day K) for ice,
* change the function to use SI base units

What is the total amount of melt in mm over all days

In [ ]:
melt["melt (mm)"].sum()

## Spatial distributed version

We take actual weather data from geosphere over the Pasterze region.

![INCA data auschnitt](figures/Pasterze.png)

Load the netcdf dataset

In [ ]:
inca_pasterze = xr.open_dataset(
    "/workspaces/Modellieren in der Physischen Geographie/data/inca_pasterze.nc"
)

In [ ]:
inca_pasterze

take onhly the Tempearature at 2 meter varaible called T2M, for one hour to get the spatial extend. 
Note the dimensions. Ther are 192 time sclices and 9 points in y and 8 points in x direction.
The INCA dataset has a resolutution of 1x1 km. Which also determines the resolution on which we can calculate the melt.

In [ ]:
temperatures = inca_pasterze["T2M"]
temperature_example = temperatures.sel(time="2023-07-01T00:00:00")
temperature_example.plot()

In [ ]:
type(temperature_example)

In [ ]:
melt_amounts_distributed = degree_day_model(temperature_example, degree_day_factor=ddf)
melt_amounts_distributed.name = "melt (mm)"

In [ ]:
melt_amounts_distributed.plot()

## Now with space and time

We take the donwloaded data which as 192 time slices store. So 8 days of hourly data.

In [ ]:
tempeartures = inca_pasterze["T2M"]
tempeartures

In [ ]:
melt_result = degree_day_model(temperatures, degree_day_factor)
melt_result.name = "melt (mm)"

In [ ]:
melt_result

Thats the result. Now we can plot and animate the result.

Basically this an open street map as a background and then makes a plot for each timestep. The result is then saved as gif.
It takes about a minute to run.

In [33]:
# Initialize the figure
fig = plt.figure(figsize=(10, 10))
osm_tiles = OSM()
ax = plt.axes(projection=osm_tiles.crs)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=":")

# Pre-calculate extent to avoid recalculating it each frame
lat_min, lat_max = np.min(melt_result.lat.values), np.max(melt_result.lat.values)
lon_min, lon_max = np.min(melt_result.lon.values), np.max(melt_result.lon.values)
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# Add base map once (assumes all slices have similar extents)
ax.add_image(osm_tiles, 12)

# Prepare colorbar (setup with dummy data)
sm = plt.cm.ScalarMappable(cmap="viridis", norm=plt.Normalize(vmin=0, vmax=150))
sm._A = []  # dummy data for ScalarMappable
cbar = plt.colorbar(sm, ax=ax, shrink=0.5, aspect=20, label="Melt Rate (mm)")


# The update function for the animation
previous_collections = []  # Store previous plot references


def update(frame):
    global previous_collections  # Refer to the global list
    # Remove previous collections
    for collection in previous_collections:
        collection.remove()
    previous_collections.clear()  # Clear the list after removal

    data_slice = melt_result.isel(time=frame)
    time_str = str(data_slice.time.values)
    new_collection = ax.pcolormesh(
        data_slice.lon,
        data_slice.lat,
        data_slice.values,
        transform=ccrs.PlateCarree(),
        cmap="viridis",
        alpha=0.5,
    )
    previous_collections.append(new_collection)  # Add new plot to the list
    ax.set_title(f"Melt Rate on {time_str}")


# Create animation
ani = FuncAnimation(fig, update, frames=len(melt_result.time), interval=200)

ani.save("figures/melt_rate_animation.gif", dpi=300)